<div style="border-radius: 5px; padding: 1rem; margin-bottom: 1rem">
<img src="https://www.prototypesforhumanity.com/wp-content/uploads/2022/11/LOGO_UTEC_.png" alt="Banner" width="150" />   
 </div>

# Laboratorio 5.2: Similitud de Coseno e Indice Invertido

> **Prof. Heider Sanchez**  
> **ACLs:** Ana María Accilio, Sebastián Loza

## Introducción

Este laboratorio tiene como objetivo que los estudiantes implementen un sistema de análisis y búsqueda de documentos utilizando Procesamiento de Lenguaje Natural (NLP) y una base de datos PostgreSQL. Se trabajará a partir de los Bag of Words generados en el laboratorio 5.1, agregando dos funcionalidades principales: búsquedas rankeadas mediante similitud de coseno e índices invertidos para una recuperación eficiente de documentos.


### Objetivos
- Implementar la lectura de Bags of Words desde PostgreSQL utilizando Python.
- Desarrollar un sistema de búsqueda de documentos similares mediante similitud de coseno que:
  - Procese consultas en lenguaje natural
  - Filtre documentos relevantes según palabras clave
  - Ordene resultados por relevancia mediante similitud de coseno
- Diseñar e implementar un índice invertido para:
  - Optimizar las búsquedas booleanas
  - Mejorar el rendimiento de la similitud de coseno
  - Reducir el tiempo de recuperación de documentos

### Requisitos previos

- Haber cargado la tabla de noticias en PostgreSQL y generado el bag of word.
- Tener instalado las dependencias de NLTK en Python.
- Completar la función para conectarte a PostgreSQL y leer los datos:

In [ ]:
import psycopg2
import pandas as pd

def connect_db():
    conn = psycopg2.connect(
        dbname="<DB>",
        user="<USER>",
        password="<PASSWORD>",
        host="<HOST>"
    )
    return conn

def fetch_data():
    conn = connect_db()
    query = "SELECT id, contenido, bag_of_words FROM noticias;"
    df = pd.read_sql(query, conn)
    df['bag_of_words'] = df['bag_of_words'].apply(json.loads)
    conn.close()
    return df

noticias_df = fetch_data()

## 1. (5 puntos) Bag of Words y Similitud de Coseno 

El proceso de búsqueda se realiza en dos etapas:

1. **Filtrado inicial**:
   - Recibe una consulta en texto natural
   - Procesa la consulta para extraer palabras clave
   - Utiliza operadores OR en SQL para recuperar documentos que contengan al menos una palabra clave

2. **Ordenamiento por relevancia**:
   - Convierte los Bag of Words de los documentos filtrados en vectores numéricos (usando solo la frecuencia)
   - Calcula la similitud de coseno entre la consulta y cada documento
   - Ordena los resultados por similitud descendente
   - Retorna los top-k documentos más relevantes

In [ ]:
# Solución Aquí

def search(query, top_k=5):
    pass

####  Pruebas funcionales

In [ ]:
test_queries = [
    "¿Cuáles son las últimas innovaciones en la banca digital y la tecnología financiera?",
    "evolución de la inflación y el crecimiento de la economía en los últimos años",
    "avances sobre sostenibilidad y energías renovables para el medio ambiente"
]

for query in test_queries:
    results = search(query, top_k=3)
    print(f"Probando consulta: '{query}'")
    for _, row in results.iterrows():
        print(f"\nID: {row['id']}")
        print(f"Similitud: {row['similarity']:.3f}")
        print(f"Texto: {row['contenido'][:200]}...")
    print("-" * 50)

## 2. (5 puntos) Construcción del Indice Invertido 

A partir de los  `bag of words` almacenados en la base de datos, se debe construir un índice invertido y conservarlo en un diccionario de Python para su eficiente recuperación.

In [ ]:
class InvertedIndex:
    def __init__(self):
        self.index = {}
        self.idf = {}
        self.length = {}

    def build_from_db(self):
        # Leer desde PostgreSQL todos los bag of words
        # Construir el índice invertido, el idf y la norma (longitud) de cada documento
        
        """
        indice  = {
            "word1": [("doc1", tf1), ("doc2", tf2), ("doc3", tf3)],
            "word2": [("doc2", tf2), ("doc4", tf4)],
            "word3": [("doc3", tf3), ("doc5", tf5)],
        } 
        idf  = {
            "word1": 3,
            "word2": 2,
            "word3": 2,
        } 
        length = {
            "doc1": 15.5236,
            "doc2": 10.5236,
            "doc3": 5.5236,
        }
        """
        pass
    
    def L(self, word):
        return self.index.get(word, [])
  
    def cosine_search(self, query, top_k=5):  
        score = {}
        # No es necesario usar vectores numericos del tamaño del vocabulario
        # Guiarse del algoritmo visto en clase
        # Se debe calcular el tf-idf de la query y de cada documento
        
        # TODO
        
        # Ordenar el score resultante de forma descendente
        result = sorted(score.items(), key= lambda tup: tup[1], reverse=True)
        # retornamos los k documentos mas relevantes (de mayor similitud a la query)
        return result[:top_k] 

## 3. (4 puntos) Consultas Booleanas usando el indice invertido

Implementar búsquedas booleanas utilizando el índice invertido construido anteriormente. La búsqueda debe:

- Soportar los operadores básicos:
    - AND: intersección de documentos
    - OR: unión de documentos
    - AND-NOT: diferencia de documentos
- Procesar consultas como:
    - "sostenibilidad AND ambiente AND renovable"
    - "tecnología AND (banca OR finanzas)"
    - "economía AND-NOT inflación"    

####  Pruebas funcionales

In [ ]:
idx = InvertedIndex()
idx.build_from_db()

def AND(list1, list2):
    # Implementar la intersección de dos listas O(n +m)
    pass

def OR(list1, list2):
    # Implementar la unión de dos listas O(n +m)
    pass

def AND_NOT(list1, list2):
    # Implementar la diferencia de dos listas O(n +m)
    pass

# Prueba 1
result = AND(idx.L("sostenibilidad"), AND(idx.L("ambiente"), idx.L("renovables")))
print("sostenibilidad AND ambiente AND renovable: ", idx.showDocuments(result))

# Prueba 2
result = AND(idx.L("tecnología"), OR(idx.L("banca"), idx.L("finanzas")))
print("tecnología AND (banca OR finanzas): ", idx.showDocuments(result))

# Prueba 3
result = AND_NOT(idx.L("economía"), idx.L("inflación"))
print("economía AND-NOT inflación: " , idx.showDocuments(result))

## 4. (6 puntos) Similitud de coseno usando el indice invertido
Implementar búsqueda por similitud de coseno aprovechando el índice invertido:

- Proceso de búsqueda:
    - Recibe una consulta en lenguaje natural y un parámetro top_k
    - Utiliza el índice invertido para identificar documentos candidatos
    - Calcula similitud de coseno solo con los documentos relevantes utilizando los pesos TF-IDF
    - Retorna los top-k documentos más similares

<img src="imagenes/image-20250506-162949.png" width="500" align="" />

####  Pruebas funcionales

In [ ]:
test_queries = [
    "¿Cuáles son las últimas innovaciones en la banca digital y la tecnología financiera?",
    "evolución de la inflación y el crecimiento de la economía en los últimos años",
    "avances sobre sostenibilidad y energías renovables para el medio ambiente"
]

for test in test_queries:    
    results = idx.cosine_search(test['query'], test['top_k'])
    print(f"Top {test['top_k']} documentos más similares:")    
    for doc_id, score in results:
        print(f"Doc {doc_id}: {score:.3f}: ", idx.showDocument(doc_id))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0cac3f27-ab57-45e1-94d7-eb1f84dca7ec' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>